In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.metrics import accuracy_score, classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
import xgboost as xgb
import warnings
warnings.filterwarnings("ignore")

# Load dataset
data = pd.read_csv("diabetes_prediction_dataset.csv")

# Handle categorical variables
data = pd.get_dummies(data, columns=["gender", "smoking_history"], drop_first=True)

# Define features and target variable
X = data.drop("diabetes", axis=1)
y = data["diabetes"]

# 80-20 split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Cross-validation setup
kf = KFold(n_splits=5, shuffle=True, random_state=42)

def evaluate_model(model, X, y):
    scores = cross_val_score(model, np.array(X), np.array(y), cv=kf, scoring='accuracy')  
    return scores.mean()


# ID3 (Entropy)
id3_clf = DecisionTreeClassifier(
    criterion="entropy",
    max_depth=5,
    random_state=42
)
id3_clf.fit(X_train, y_train)
id3_preds = id3_clf.predict(X_test)

id3_acc = accuracy_score(y_test, id3_preds)
id3_cross_val = evaluate_model(id3_clf, X, y)

print("ID3 Decision Tree Accuracy:", id3_acc)
print("\nClassification Report:")
print(classification_report(y_test, id3_preds))

# CART (Gini)
cart_clf = DecisionTreeClassifier(
    criterion="gini",
    max_depth=5,
    random_state=42
)
cart_clf.fit(X_train, y_train)
cart_preds = cart_clf.predict(X_test)

cart_acc = accuracy_score(y_test, cart_preds)
cart_cross_val = evaluate_model(cart_clf, X, y)

print("CART (Gini) Decision Tree Accuracy:", cart_acc)
print("\nClassification Report:")
print(classification_report(y_test, cart_preds))

# Random Forest
rf_clf = RandomForestClassifier(
    n_estimators=100,
    max_depth=5,
    random_state=42
)
rf_clf.fit(X_train, y_train)
rf_preds = rf_clf.predict(X_test)

rf_acc = accuracy_score(y_test, rf_preds)
rf_cross_val = evaluate_model(rf_clf, X, y)

print("Random Forest Accuracy:", rf_acc)
print("\nClassification Report:")
print(classification_report(y_test, rf_preds))

# Gradient Boosting
gb_clf = GradientBoostingClassifier(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=5,
    random_state=42
)
gb_clf.fit(X_train, y_train)
gb_preds = gb_clf.predict(X_test)

gb_acc = accuracy_score(y_test, gb_preds)
gb_cross_val = evaluate_model(gb_clf, X, y)

print("Gradient Boosted Trees Accuracy:", gb_acc)
print("\nClassification Report:")
print(classification_report(y_test, gb_preds))

# XGBoost
xgb_clf = xgb.XGBClassifier(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=5,
    random_state=42,
    use_label_encoder=False
)

xgb_clf.fit(X_train, y_train)
xgb_preds = xgb_clf.predict(X_test)

xgb_acc = accuracy_score(y_test, xgb_preds)
xgb_cross_val = evaluate_model(
    xgb.XGBClassifier(
        n_estimators=100,
        learning_rate=0.1,
        max_depth=10,
        random_state=42
    ),
    X, y
)


print("XGBoost Accuracy:", xgb_acc)
print("\nClassification Report:")
print(classification_report(y_test, xgb_preds))

# Compare model performance
models = ["ID3 (Entropy)", "CART (Gini)", "Random Forest", "Gradient Boosting", "XGBoost"]
accuracies = [id3_acc, cart_acc, rf_acc, gb_acc, xgb_acc]
cross_val_scores = [id3_cross_val, cart_cross_val, rf_cross_val, gb_cross_val, xgb_cross_val]

results_df = pd.DataFrame({"Model": models, "Test Accuracy": accuracies, "Cross-Val Accuracy": cross_val_scores})

print(results_df)

# Visualization
plt.figure(figsize=(10, 5))
sns.barplot(x=results_df["Model"], y=results_df["Test Accuracy"], palette="viridis")

plt.ylim(0.7, 1)
for index, value in enumerate(results_df["Test Accuracy"]):
    plt.text(index, value + 0.005, f"{value:.3f}", ha='center', fontsize=12)

plt.xlabel("Model", fontsize=12)
plt.ylabel("Test Accuracy", fontsize=12)
plt.title("Comparison of Decision Tree Models", fontsize=14)
plt.xticks(rotation=45)
plt.grid(axis="y", linestyle="--", alpha=0.7)

plt.show()


: 